## 評估指標與其用途

在機器學習模型做出預測之後，用以衡量預測優劣的指標被稱為<b>評估指標</b>。機器學習分類任務的評估指標有<b>分類精度</b>和迴歸問題的<b>均方根誤差</b>..等等，評估指標可以提供明確客觀的依據，我們得以衡量機器學習流程的優劣，以下是幾個常見的情境:
<li>以同樣資料來源但不同的縮放方法產生的資料，訓練同一個機器學習模型，使用評估指標能衡量用於此機器學習模型最好的一種縮放方法
<li>以同樣資料、訓練多個機器學習模型，使用評估指標能衡量在使用此資料來源，使用哪一個機器學習模型的表現最佳
<li>以同樣資料、訓練一種機器學習模型，但使用多組不同的參數，使用評估指標能衡量哪一組機器學習的參數最佳

### 目標

此筆記本中，將以Python實現四個標準預測
<li>如何實現分類準確性。
<li>如何實現和解釋混淆矩陣。
<li>如何實現回歸的平均絕對誤差。
<li>如何實現回歸的均方根誤差。

### 1. [分類]分類準確性-Classification Accuracy
最直觀的機器學習分類任務的評估指標，是計算其所有預測當中成功預測的百分比。分類準確性的結果是0-1之間的數字，0代表完全沒有成功的預測；1代表預測全部成功。分類準確性需要有正確的label以及機器學習預測的label來做評估。以下為公式:

\begin{align}
accuracy = \frac{correct prediction}{total prediction}
\end{align}

##### Python implementation

In [1]:
def accuracy_metric(actual_labels, predicted_labels):
    """
    分類準確性，計算所有預測中被正確預測的比例
    
    args:
    actual_labels(list):正確的標籤(label) ,也被稱為 groung truth.
    predicted_labels(list):機器學習模型所預測的標籤(label)
    
    return:
    一個介於0-1之間的數字，代表被正確預測的標籤的比例  
    """
    correct = 0    
    for i in range(len(actual_labels)):
        if predicted_labels[i] == actual_labels[i]:
            correct += 1
    return correct / len(actual_labels)        

In [9]:
#實際的標籤
actual_labels = [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]
#機器學習預測的標籤
predicted_labels = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

#計算機器學習分類預測的準確度
accuracy_metric(actual_labels, predicted_labels)

0.8333333333333334

### 分類準確性之缺點
分類準確性是很直觀的評估指標，我們得以知道預測命中的百分比。但是當遇到一些情形，分類準確性會失去其意義或表現得近乎假象。假設我們使用的機器學習模型是建立在二元分類的任務上(預測<b>是</b>或<b>否</b>)，但表現極差，對所有的分類預測都只預測為0。但是當遇到這樣的資料集；有高達90%的資料label為0，僅有10%為1，即使我們的機器學習模型只會預測全部標籤皆為0，但我們的分類準確性還是高達90%的準確率，讓我們誤以為機器學習模型的表現沒有問題。也因此當遇到分佈不平衡的資料集時(某一些類別多於其他類別)，時常分類準確性會誤導我們對機器學習模型表現的認知。

### 2. [分類]混淆矩陣-Confushion matrix
評估機器學習模型的分類性能的更好方法是使用混淆矩陣。混淆矩陣的概念是用來計算某類別被錯誤預測為其他類的次數。混淆矩陣是二維矩陣且寬等於高，也就是說假設資料總共有N個類別，則用來評估該資料之預測結果的混淆矩陣維度為NxN。一個簡單的理解方式是將混淆矩陣看成一個table，每一行(row)，代表實際類別n，而該行的每一個欄位(column)該實際類別被預測為某類別的次數。

##### Python implementation

In [14]:
def confushion_matrix(actual_labels, predicted_labels):
    """
    混淆矩陣，計算所有被正確預測以及被錯誤預測為其他類別的次數總計
    
    args:
    actual_labels(list):正確的標籤(label) ,也被稱為 groung truth.
    predicted_labels(list):機器學習模型所預測的標籤(label)
    
    return:
    unique:一個所有類別的列表
    matrix:一個混淆矩陣  
    """
    
    unique = set(actual_labels)#所有類別的列表
    
    matrix = [list() for x in range(len(unique))] 
    for i in range(len(unique)):
        matrix[i] = [0 for j in range(len(unique))]
        
    lookup = {}
    
    for i, value in enumerate(unique):
        lookup[value] = i
        
    for i in range(len(actual_labels)):
        x = lookup[actual_labels[i]]
        y = lookup[predicted_labels[i]]
        matrix[x][y] += 1
        
    return unique, matrix

In [22]:
#實際的標籤
actual_labels =    [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]
#機器學習預測的標籤
predicted_labels = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

#計算混淆矩陣
#而實際上是0但是被預測為1的次數是0，實際標籤與預測標籤同為0的次數為9次。
#實際上是1但是被預測為0的次數為2，實際標籤與預測標籤同為1的次數為1次。
lookup, matrix = confushion_matrix(actual_labels,
                                   predicted_labels)
matrix

[[9, 0], [2, 1]]

In [19]:
#視覺化混淆矩陣
def print_confusion_matrix(unique, matrix):
    print('(A)' + ' '.join(str(x) for x in unique))
    print('(P)---')
    for i, x in enumerate(unique):
        print("%s| %s" % (x, ' '.join(str(x) for x in matrix[i])))

print_confusion_matrix(lookup, matrix)

(A)0 1
(P)---
0| 9 0
1| 2 1


##### Sklearn implementation

In [21]:
from sklearn.metrics import confusion_matrix

confusion_matrix(actual_labels, predicted_labels)

array([[9, 0],
       [2, 1]], dtype=int64)

### 3. [迴歸]平均絕對誤差-Mean Absolute Error (MAE)
平均絕對誤差測量一組預測中的誤差的平均幅度，而不考慮它們的正負。是測試樣本中預測和實際觀察之間絕對差異的平均值。公式如下:
\begin{align}
MAE = \frac{1}{n}*\frac{\sum_{i=1}^n|y_i - \hat{y_i}|}{n - 1}
\end{align}
另外如果MAE未採用絕對值（未消除誤差的符號），則平均誤差變為平均偏差誤差（MBE），並且通常用於測量平均模型偏差。 MBE可以傳達有用的信息，但應該謹慎解釋，因為正面和負面的錯誤都會被取消。

In [25]:
def mae_metric(actual_values, predicted_values):
    abs_error = 0
    for i in range(len(actual_values)):
        abs_error += abs(actual_values[i] - predicted_values[i])
    return abs_error / len(actual_values)

In [27]:
# Test RMSE
actual = [0.1, 0.2, 0.3, 0.4, 0.5]
predicted = [0.11, 0.19, 0.29, 0.41, 0.5]
mae = mae_metric(actual, predicted)
mae

0.007999999999999993

### 4. [迴歸]均方根誤差-Root-Mean-Square Error (RMSE)
RMSE測量誤差的平均幅度。 它是預測和實際觀察之間平均差異平均值的平方根。公式如下:
\begin{align}
RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{y_i - \hat{y_i}}{\sigma_i}\Big)^2}}
\end{align}

##### Python implementation

In [5]:
from math import sqrt

def rmse_metric(actual_values, predicted_values):
    """
    均方根誤差，計算預測和實際觀察之間平均差異平均值的平方根
    
    args:
    actual_labels(list):正確的標籤(label) ,也被稱為 groung truth.
    predicted_labels(list):機器學習模型所預測的標籤(label)
    
    return:
    rmse:均方根誤差
    """
    diff_sum = 0
    for i in range(len(actual_values)):
        diff_sum += (actual_values[i] - predicted_values[i])**2
        
    return sqrt(diff_sum / len(actual_values))

In [7]:
actual = [0.1, 0.2, 0.3, 0.4, 0.5]
predicted = [0.11, 0.19, 0.29, 0.41, 0.5]
rmse_metric(actual, predicted)

0.00894427190999915

##### Numpy implementation

In [16]:
import numpy as np

def rmse_metric(actual_values, predicted_values):
    """
    均方根誤差，計算預測和實際觀察之間平均差異平均值的平方根
    
    args:
    actual_labels(list):正確的標籤(label) ,也被稱為 groung truth.
    predicted_labels(list):機器學習模型所預測的標籤(label)
    
    return:
    rmse:均方根誤差
    """
    mse = np.mean((np.array(actual_values) - np.array(predicted_values))**2)
        
    return np.sqrt(mse)

In [17]:
actual = [0.1, 0.2, 0.3, 0.4, 0.5]
predicted = [0.11, 0.19, 0.29, 0.41, 0.5]
rmse_metric(actual, predicted)

0.00894427190999915

### MAE 與 RMSE之比較
相似之處：MAE和RMSE都變量為單位表示平均模型預測誤差。 兩個度量都可以在0到∞的範圍內，並且正負無關緊要。 它們是負面導向的分數，這意味著較低的值更好。

差異：取平均誤差的平方根對RMSE有一些有趣的影響。 由於誤差在平均之前是平方的，因此RMSE對大誤差給出相對較高的權重。 這意味著當特別不希望出現大錯誤時RMSE更有用。 下面的三個表格顯示了MAE穩定且RMSE增加的示例，因為與誤差幅度的頻率分佈和方差的幅度差異。
<img src='images\section-3\rmse_mae_compare_1.jpg' style='width:700px;'>

RMSE不一定隨著誤差的變化而增加，RMSE隨方差的誤差幅度和頻率分佈而增加。
為了演示，請考慮下表中的案例4和案例5。 案例4具有相同數量的0和5的測試錯誤，案例5具有相同數量的3和4的測試錯誤。案例4中的錯誤差異更大但案例4和案例5的RMSE相同。也就是說當方差的頻率分佈較為集中時，RMSE的數值近於MAE，但是隨著頻率分佈較離散時會高於MAE。另外即使頻率分佈趨於集中，但是有極端值產生時，RMSE還是會大幅增加。
<img src='images\section-3\rmse_mae_compare_2.jpg' style='width:700px;'>

### 結論
RMSE具有更多懲罰大誤差值的好處，因此在某些情況下可能更合適，例如，當我們希望誤差的幅度與頻率散布盡可能的小時我們會使用RMSE，但如果誤差的幅度大小在我們看來並不在意(不管誤差10或是誤差100都是一樣差)那麼MAE更合適。從解釋性的角度來看，MAE顯然更讓人容易理解。 RMSE不僅僅描述平均誤差，而且還有其他更難以梳理的含義。另一方面，RMSE相對於MAE的一個明顯優勢是RMSE避免使用絕對值，這在許多數學計算中是不可取的。

## More to come.....
評估指標多種多樣，未來將補充更多重要的指標，例如:
<li>precision
<li>recall
<li>f1-score
<li>AUC